# Notebook 2 — Output Contract (JSON) + Validation + Repair (Prod yaklaşımı)

Bu notebook, LLM çıktısını üretime yakın hale getirir:
- JSON-only contract
- Parse
- Schema doğrulama (Pydantic)
- Hatalı çıktıda repair + retry



## Setup

Configuration with azure openai api key

In [ ]:
# Lokal/Colab için (gerekliyse) kurulum:
%pip -q install -U langchain-core langchain-openai langchain-google-genai pydantic tiktoken python-dotenv pandas matplotlib

In [ ]:
import os, json, re
from typing import Dict, Any

def get_llm():
    # Sağlayıcı seçimi: OpenAI -> Gemini
    openai_key = os.getenv('OPENAI_API_KEY')
    google_key = os.getenv('GOOGLE_API_KEY')

    if openai_key:
        from langchain_openai import ChatOpenAI
        model = os.getenv('OPENAI_MODEL', 'gpt-4o-mini')
        llm = ChatOpenAI(model=model, temperature=0.1)
        return llm, f'OpenAI ({model})'

    if google_key:
        from langchain_google_genai import ChatGoogleGenerativeAI
        model = os.getenv('GEMINI_MODEL', 'gemini-1.5-pro')
        llm = ChatGoogleGenerativeAI(model=model, temperature=0.1, api_key=google_key)
        return llm, f'Google Gemini ({model})'

    raise RuntimeError(
        'API key bulunamadı. Lütfen ortam değişkeni tanımlayın:\n'
        '- OpenAI için: OPENAI_API_KEY\n'
        '- Gemini için: GOOGLE_API_KEY\n'
        'Opsiyonel: OPENAI_MODEL / GEMINI_MODEL'
    )

llm, provider = get_llm()
print('✅ LLM hazır:', provider)

def llm_text(prompt: str) -> str:
    resp = llm.invoke(prompt)
    return getattr(resp, 'content', str(resp)).strip()

def strip_fences(s: str) -> str:
    s = s.strip()
    s = re.sub(r'^```json\s*', '', s, flags=re.IGNORECASE)
    s = re.sub(r'^```\s*', '', s)
    s = re.sub(r'\s*```$', '', s)
    return s.strip()

In [ ]:
# Mini veri seti (email/ticket) — demo için
EMAILS = [
    {'id': 'E1', 'text': 'Kargom hâlâ gelmedi. 7 gündür bekliyorum. Acil çözüm istiyorum!', 'notes': 'Gecikme + yüksek aciliyet'},
    {'id': 'E2', 'text': 'Ürün kırık geldi. Değişim yapabilir miyiz?', 'notes': 'Hasarlı ürün'},
    {'id': 'E3', 'text': 'İade sürecini nasıl başlatabilirim? Kutuyu attım ama ürün duruyor.', 'notes': 'İade + edge-case (kutusuz)'},
    {'id': 'E4', 'text': 'Kartımdan iki kez çekim yapılmış görünüyor. Lütfen hemen kontrol edin.', 'notes': 'Faturalama + yüksek aciliyet'},
    {'id': 'E5', 'text': 'Ürününüzün kullanım kılavuzunu paylaşır mısınız?', 'notes': 'Bilgi talebi (low)'},
]
len(EMAILS)

## 1) JSON Schema (Pydantic)

In [ ]:
from pydantic import BaseModel, Field, ValidationError
from typing import Literal

class TriageOut(BaseModel):
    category: str = Field(min_length=1)
    urgency: Literal['low','medium','high']
    reason: str = Field(min_length=1, max_length=240)

## 2) Contract tanımı

In [ ]:
SCHEMA = (
    'Return ONLY valid JSON with exactly these keys:\n'
    '{\n'
    '  "category": "string",\n'
    '  "urgency": "low|medium|high",\n'
    '  "reason": "string (max 1 sentence)"\n'
    '}\n'
    'No extra text, no markdown, JSON only.'
)

## 3) Contract yoksa ne olur?

Genelde açıklama/metin döner; parse zorlaşır.

In [ ]:
def prompt_no_contract(email_text: str) -> str:
    return (
        'Classify this email: category and urgency.\n'
        'Explain briefly.\n\n'
        'Email:\n' + email_text
    )

raw = llm_text(prompt_no_contract(EMAILS[0]['text']))
print(raw)

## 4) Contract ile tekrar

In [ ]:
def prompt_with_contract(email_text: str) -> str:
    return (
        'You are a customer support triage assistant.\n'
        'Classify the email into a category and urgency.\n\n'
        + SCHEMA + '\n\n'
        + 'Email:\n' + email_text
    )

raw = llm_text(prompt_with_contract(EMAILS[0]['text']))
print(raw)

## 5) Parse + Validate

In [ ]:
import json
from typing import Dict, Any

def parse_json(raw: str) -> Dict[str, Any]:
    return json.loads(strip_fences(raw))

def validate(obj: Dict[str, Any]) -> TriageOut:
    return TriageOut.model_validate(obj)

raw = llm_text(prompt_with_contract(EMAILS[0]['text']))
obj = parse_json(raw)
validated = validate(obj)
validated

## 6) Repair / Retry

Üretimde çok kullanılan yaklaşım:
- İlk deneme
- Parse/validate başarısızsa: **çıktıyı** tekrar modele verip sadece JSON formatına zorlamak
- 1–2 retry ile stabil hale getirmek

In [ ]:
REPAIR_PROMPT = (
    'You are a strict JSON formatter.\n'
    'Fix the following output to match this JSON schema exactly and return JSON only.\n\n'
    'Schema:\n{\n  "category": "string",\n  "urgency": "low|medium|high",\n  "reason": "string (max 1 sentence)"\n}\n\n'
    'Bad output:\n{bad_output}'
)

def triage_with_retry(email_text: str, max_retries: int = 1) -> TriageOut:
    last_raw = llm_text(prompt_with_contract(email_text))
    for attempt in range(max_retries + 1):
        try:
            obj = parse_json(last_raw)
            return validate(obj)
        except Exception as e:
            if attempt >= max_retries:
                print('❌ Failed. Last raw:\n', last_raw)
                raise
            last_raw = llm_text(REPAIR_PROMPT.format(bad_output=last_raw))

out = triage_with_retry(EMAILS[2]['text'], max_retries=1)
out

## 7) Batch: kaç tanesi retry ile toparlandı?

In [ ]:
import pandas as pd

rows = []
for e in EMAILS:
    try:
        out = triage_with_retry(e['text'], max_retries=1)
        rows.append({'id': e['id'], 'ok': True, 'category': out.category, 'urgency': out.urgency})
    except Exception:
        rows.append({'id': e['id'], 'ok': False, 'category': None, 'urgency': None})

pd.DataFrame(rows)

## 8) Egzersiz (3–5 dk)

1) `reason` tek cümle mi? Regex ile kontrol ekleyin.
2) `category` için izinli liste tanımlayıp standardize edin.

➡️ Sonraki notebook: token/context maliyeti + CoT kalite/maliyet tradeoff’u.